# Asynchronous Initialization 

#### RTM Cell

$\alpha_{n} = 0.032*(v+52)/(1-e^{-(v+52)/5})$

$\beta_{n} = 0.5*e^{-(v+57)/40}$

$\alpha_{m} = 0.32*(v+54)/(1-e^{-(v+54)/4})$

$\beta_{m} = 0.28*(v+27)/(e^{(v+27)/5}-1)$

$\alpha_{h} = 0.128*e^{-(v+50)/18}$

$\beta_{h} = 4./(1+e^{-(v+27)/5})$

In [1]:
import numpy as np
import logging

from bmtk.builder.networks import NetworkBuilder

n_E =30

net = NetworkBuilder('network')
net.add_nodes(N=n_E, pop_name='Exc',     # N = number of excitatory cells
        model_type='biophysical',
        model_template='hoc:RTMExcCell', # RTMExcCell hoc definition
        morphology='blank.swc')

c:\users\strou\anaconda3\envs\py2env\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Initialize "frozen noise" for current clamp and RTM synapse strength

In [2]:
i_ext= np.random.rand(n_E,1)*0.1+0.25

#create array of start times for heterogeneous initialization
start = np.random.rand(n_E,1)* 25 # array of 30 with times from 0 to 25
print(start)


[[ 0.61012523]
 [22.3593201 ]
 [23.02129682]
 [ 4.30644742]
 [10.9826885 ]
 [ 1.53455052]
 [ 2.58276901]
 [21.48073134]
 [17.1535747 ]
 [16.2540257 ]
 [ 3.2640467 ]
 [ 8.20755167]
 [17.8493559 ]
 [24.80628412]
 [17.85006565]
 [14.46999368]
 [11.46029854]
 [13.29252038]
 [21.73661013]
 [ 2.23213208]
 [19.79586685]
 [ 7.69193825]
 [ 5.83109892]
 [20.23210291]
 [15.55680751]
 [ 5.33391483]
 [10.60910746]
 [12.05219427]
 [ 1.69229427]
 [ 1.58189327]]


In [3]:
net.build()
net.save_nodes(output_dir='network')

In [4]:
from bmtk.analyzer import nodes_table
print('network_nodes.h5')
nodes_table(nodes_file='network/network_nodes.h5', population='network')

network_nodes.h5


c:\users\strou\anaconda3\envs\py2env\lib\site-packages\bmtk-0.0.8-py2.7.egg\bmtk\simulator\utils\config.py:4: UserWarning: Please use bmtk.simulator.core.simulation_config instead.
  warnings.warn('Please use bmtk.simulator.core.simulation_config instead.')


,node_id,node_type_id
0,0,100
1,1,100
2,2,100
3,3,100
4,4,100
5,5,100
6,6,100
7,7,100
8,8,100
9,9,100


In [5]:
from bmtk.analyzer import node_types_table
print('network_node_types.h5')
node_types_table(node_types_file='network/network_node_types.csv', population='network')

network_node_types.h5


,node_type_id,model_type,morphology,model_template,pop_name
0,100,biophysical,blank.swc,hoc:RTMExcCell,Exc


Add excitatory from each RTM cell to every other RTM cell in the network and save

In [6]:
# add 1 synaptic connection from each cell to every other cell avoiding self connections
def like2like(source, target, n):
    """A simple function for setting # of synaptic connections based on pop-name attribute. """
    if source['node_id'] == target['node_id']:
        # No autapses
        return 0

    # 1 connection otherwise
    return n


def rand_syn_weight(source, target, min_weight, max_weight):
    return np.random.uniform(min_weight, max_weight)


conn = net.add_edges(source={'pop_name': 'Exc'}, target={'pop_name': 'Exc'},
              connection_rule=like2like, # Note that we are passing in the function name but not calling it
              connection_params={'n': 1}, # 1 synaptic connection to all other cells
              target_sections=['soma'],
              dynamics_params='PING_ExcToExc.json',
                     
              model_template='Exp2Syn',
              delay=0)

conn.add_properties('syn_weight',
                    rule=rand_syn_weight,
                    rule_params={'min_weight': 0.00625, 'max_weight': 0.00825},
                    dtypes=np.float)

# Build and save connections



In [7]:
net.build()
net.save_edges(output_dir='network')

We now need to create an initial spike for each of our cells with varying phase

In [8]:
from bmtk.utils.reports.spike_trains import SpikeTrains

spikes = SpikeTrains(population='Exc')
spikes.add_spikes(node_ids=range(30), 
                    timestamps=start)
spikes.to_sonata('input/Asynchronous.h5')

The RTM neurons also have a current clamp input that was defined earlier in our "frozen noise"

In [9]:
import h5py


dts = [1]
gids = "all"

#hf.close() # Uncomment this line temporarily if you receive 'file already open error'

hf = h5py.File("input/amps.h5", 'w')

hf.create_dataset("amplitudes", data=[i_ext])
hf.create_dataset("gids", data=gids)
hf.create_dataset("dts", data=dts)

hf.close()

### Building the Simulation Environment

In [12]:
from bmtk.utils.sim_setup import build_env_bionet

build_env_bionet(base_dir='.',
                 network_dir='network',
                 tstop=800.0, dt=0.1,
                 report_vars=['v'],     # Record membrane potential and calcium (default soma)
                 spikes_inputs=[('Exc',   # Name of population which spikes will be generated for
                                'input/Asynchronous.h5')],
                 include_examples=False,    # Copies components files
                 compile_mechanisms=False   # Will try to compile NEURON mechanisms
                )

### Running the Simulation

In [14]:
from bmtk.simulator import bionet


conf = bionet.Config.from_json('simulation_config.json')
conf.build_env()
net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

WindowsError: [Error 32] The process cannot access the file because it is being used by another process: u'C:\\Users\\strou\\Documents\\Git\\Borgers-Examples-in-BMTK\\24-RTM_Heterogeneous\\output\\log.txt'

In [ ]:
from bmtk.analyzer.spike_trains import plot_raster, plot_rates_boxplot

plot_raster(config_file='simulation_config.json', group_by='pop_name')